In [2]:
!pip install cupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 32.4 MB/s 
ERROR: Operation cancelled by user


In [2]:
import numpy as np


def rle_encode(img):
    '''
    this function is adapted from
    https://www.kaggle.com/code/stainsby/fast-tested-rle/notebook
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

import cupy as cp

def mask2rle(msk, thr=0.5):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    msk    = cp.array(msk)
    pixels = msk.flatten()
    pad    = cp.array([0])
    pixels = cp.concatenate([pad, pixels, pad])
    runs   = cp.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def test1():
    a = np.random.rand(12,10)
    shape0 = np.array([7, 6])
    resize = np.array([12, 10])
    diff = resize - shape0
    pad0 = diff[0]
    pad1 = diff[1]
    pady = [pad0//2, pad0//2 + pad0%2]
    padx = [pad1//2, pad1//2 + pad1%2]
    print(f'b=a[{pady[0]}:{-pady[1]},{padx[0]}:{-padx[1]}]')
    b = a[pady[0]:-pady[1], padx[0]:-padx[1]]
    d = np.transpose(b)
    
    print(f'a:{a.shape};b:{b.shape};d:{d.shape}\n')

    d[d>0.4]=1
    d[d<=0.4]=0
    print(d)
    e = rle_encode(d)

    print(e)
    print(mask2rle(d))

    

if __name__ == '__main__':
    test1()

b=a[2:-3,2:-2]
a:(12, 10);b:(7, 6);d:(6, 7)

[[1. 1. 1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 0. 1. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]
 [0. 0. 1. 1. 1. 0. 1.]]
1 5 7 1 12 6 19 1 24 6 33 3 38 3 42 1
1 5 7 1 12 6 19 1 24 6 33 3 38 3 42 1
